# Simple Code-Shortening Idioms #

A collection of some very simple techniques for performing common tasks, that hopefully you will find useful.

## Conditionally Deleting A Dictionary Entry ##

You may know that you can delete a dictionary entry with

    del dict[key]

but this raises a `KeyError` exception if `key` is not present in the dictionary. So how do you delete an entry _only if it is present_ ? This

    if key in dict :
        del dict[key]
    #end if

seems unwieldy. But there is a shorter way. The `dict` class defines [a pop method](https://docs.python.org/3/library/stdtypes.html#mapping-types-dict) that removes and returns an entry with a specified key; this call also raises `KeyError` if the key is not present, unless you specify a default value that is returned in this case. So, to delete an entry if present and avoid any error from its absence, pass any dummy default value (e.g. `None`) and ignore the result:

    dict.pop(key, None)

## Conditionally Looking Up A Dictionary Entry ##

(This one seems so obvious, yet I have seen code that was clearly written without knowing it could be done this way.)

If you try to access a dictionary entry `dict[key]` that does not exist, you will get a `KeyError` exception. If you want to get the entry if it is there, but do something else if it is not, I have seen code like

    if key in dict :
        val = dict[key]
        ... do something with val ...
    else :
        ... do something else ...
    #end if

There is a more direct way. The `dict` class defines a `get()` method, which returns a default value if the key is not present. You can specify your own default value, but if not, the default for the default is `None`. _E.g._

    val = dict.get(key)
    if val != None :
        ... do something with val ...
    else :
        ... do something else ...
    #end if


## Testing Mask Bits ##

You may not have immediately noticed from reading the Python language spec, but [binary bitwise operators have higher precedence than comparisons](https://docs.python.org/3/reference/expressions.html#binary-bitwise-operations). This differs from C and most other languages that follow C conventions. You may think this could easily trip up someone transliterating expressions from C or other such languages to Python, but I am not aware of any real-world situation where this could be a problem.

What it _does_ mean is that you can write bitmask tests with fewer parentheses. For example, in C you have to write things like

    («flags» & «mask») != 0
    («flags» & «mask1») == «mask2»

but in Python these parentheses are unnecessary:

    «flags» & «mask» != 0
    «flags» & «mask1» == «mask2»


## Loading A File Into Memory ##

Opening a file read-only and loading its entire contents into memory can be done with one line:

    string_contents = open(«filename», "rt").read()

or

    binary_contents = open(«filename», "rb").read()

But what about closing the file after reading? Ah, the nice thing is, CPython manages objects by reference-counting, so once the reference count of the `file` object reaches zero, it is automatically closed.


## Avoiding The `with`-Statement ##

`with`-statements have their uses, for example for managing memory allocated by some lower-level library that is not creating Python objects. But I often see them used with files and temporary directories, e.g.

    with open(«filename», «mode») as fileobj :
        ... do something with fileobj ...
    #end with

or

    with tempfile.TemporaryDirectory() as tempdir :
        ... create and manipulate various files in tempdir ...
    #end with

The aim is that the `with`-statement guarantees that, in any situation short of process deletion or complete runtime context meltdown, the file will be closed and the temporary directory (and its contents) cleaned up.

Trouble is, this is not necessarily a good thing.

For files opened read-only, this is unnecessary anyway, because as mentioned in the previous section, the file will be automatically closed when the last reference to the file object disappears.

For files opened for writing, I always have this concern about notifying the user as far as possible about any I/O errors. Because to fail silently could lead to data loss, where the user assumes that data has been saved, but it has not. So I always like to explicitly close such files:

    fileobj = open(«filename» «writemode»)
    .... fileobj.write(«data») ...
    fileobj.close()

That way any errors on flushing that last bufferful get a chance to be reported on the `close()` call. If the program never gets that far, then the user will not be misled by seeing a `with`-statement into assuming that any meaningful cleanup is going to be done.

For temporary directories, the `with`-statement guarantees as far as possible that the temporary directory will disappear, even if an exception occurs. Trouble is, if an exception occurs, the contents of that directory could be useful for debugging purposes. So, as messy as it may appear, I like to explicitly clean up the directory myself:

    tempdir = tempfile.mkdtemp()
    ... create and manipulate various files in tempdir ...
    shutil.rmtree(tempdir)

So now the temporary files only disappear if all goes well. If you see them piling up, then there is something wrong with your code!